50% Data Modified

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('processed_friday_dataset_new.csv')

# Filter rows where flag is 1
df_filtered = df[df['Label'] == 1]

# Take 50% of the filtered data randomly
df_sample = df_filtered.sample(frac=0.5, random_state=42)

# Change a column value
df_sample['Label'] = 0

# Combine the modified rows with the unmodified ones
df_final = pd.concat([df[~df.index.isin(df_sample.index)], df_sample])

# Save the result back to a CSV file
df_final.to_csv('poisoning_bot.csv', index=False)


In [4]:
df_dataset = pd.read_csv("poisoning_bot.csv")
print(df_dataset)

         Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0               141385             9             7              553   
1                  281             2             1               38   
2               279824            11            15             1086   
3                  132             2             0                0   
4               274016             9            13             1285   
...                ...           ...           ...              ...   
1364426          10224             3             4              326   
1364427          10837             3             4              326   
1364428          15931             3             4              326   
1364429          12556             3             4              326   
1364430          11482             3             4              326   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
0                 3773.0              202                0         

In [6]:
print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())
print(df_dataset["Label"].value_counts()[[2]].sum())
print(df_dataset["Label"].value_counts()[[3]].sum())

1146610
72267
98
145456


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import graphviz
import pydotplus
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree

# For reproducible results

In [10]:
from sklearn.model_selection import train_test_split
RANDOM_STATE_SEED = 420
train, test = train_test_split(df_dataset, test_size=0.3, random_state=RANDOM_STATE_SEED)

In [12]:
train

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_17,Protocol_6,Label
358501,852,3,4,161,488.0,161,0,53.666667,92.953393,488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
205523,2224329,8,7,1128,1581.0,661,0,141.000000,222.623321,1173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
334013,7949,3,4,161,488.0,161,0,53.666667,92.953393,488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
1244653,4228930,5,3,935,412.0,935,0,187.000000,418.144712,412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
1309914,10383,3,4,326,129.0,326,0,108.666667,188.216188,112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049785,4218271,5,3,935,395.0,935,0,187.000000,418.144712,395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
178094,341,1,1,77,205.0,77,77,77.000000,0.000000,205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,0
510579,601,3,4,161,488.0,161,0,53.666667,92.953393,488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
1242184,4424753,5,4,935,700.0,935,0,187.000000,418.144712,350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0


In [14]:
test

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_17,Protocol_6,Label
421031,815,1,1,25,83.0,25,25,25.000000,0.000000,83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,0
1026751,61714,3,0,0,0.0,0,0,0.000000,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,3
882513,17585,2,0,0,0.0,0,0,0.000000,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,3
413896,251593,3,1,0,0.0,0,0,0.000000,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
189064,4094574,10,7,1148,1581.0,677,0,114.800000,210.092466,1173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341771,1439,1,1,41,127.0,41,41,41.000000,0.000000,127,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,0
1281286,4236435,5,3,935,310.0,935,0,187.000000,418.144712,310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
606144,827,3,4,161,488.0,161,0,53.666667,92.953393,488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0
188978,3450483,9,9,1213,1874.0,661,0,134.777778,209.079995,1173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0


In [16]:
print("Full dataset:\n")
print("Benign: " + str(df_dataset["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(df_dataset["Label"].value_counts()[[1]].sum()))
print("---------------")

print("Training set:\n")
print("Benign: " + str(train["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(train["Label"].value_counts()[[1]].sum()))
print("---------------")

print("Test set:\n")
print("Benign: " + str(test["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(test["Label"].value_counts()[[1]].sum()))



print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())
print(df_dataset["Label"].value_counts()[[2]].sum())
print(df_dataset["Label"].value_counts()[[3]].sum())

Full dataset:

Benign: 1146610
Malicious: 72267
---------------
Training set:

Benign: 802876
Malicious: 50723
---------------
Test set:

Benign: 343734
Malicious: 21544
1146610
72267
98
145456


In [18]:
y_train = np.array(train.pop("Label")) # pop removes "Label" from the dataframe
X_train = train.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(955101, 72)
(955101,)


In [20]:
y_test = np.array(test.pop("Label")) # pop removes "Label" from the dataframe
X_test = test.values

print(type(X_test))
print(type(y_test))
print(X_test.shape)
print(y_test.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(409330, 72)
(409330,)


In [22]:
model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0
)

hyperparameters = {
    'max_depth': [i for i in range(1, 20)]
}

In [24]:
clf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [26]:
clf.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19]},
             verbose=1)

In [28]:
print("Accuracy score on Validation set: \n")
print(clf.best_score_ )
print("---------------")
print("Best performing hyperparameters on Validation set: ")
print(clf.best_params_)
print("---------------")
print(clf.best_estimator_)

Accuracy score on Validation set: 

0.9471574212227815
---------------
Best performing hyperparameters on Validation set: 
{'max_depth': 11}
---------------
DecisionTreeClassifier(max_depth=11)


In [30]:
model = clf.best_estimator_

In [32]:
joblib.dump(model, "trained_models/decision-tree-classifier-poisoningbot.pkl")

['trained_models/decision-tree-classifier-poisoningbot.pkl']

In [34]:
predictions = model.predict(X_test)

In [36]:
predictions

array([0, 3, 3, ..., 0, 0, 0])

In [38]:
print(accuracy_score(y_test, predictions))

0.9471038037769037


In [181]:
from joblib import load
model = load("trained_models/decision-tree-classifier-poisoningbot.pkl")

In [183]:
df_dataset = pd.read_csv("ATTACKS.csv")
print(df_dataset)

y_train = np.array(df_dataset.pop("Label")) # pop removes "Label" from the dataframe
X_train = df_dataset.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

    Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0           16079             3             4              326   
1             577             2             0                0   
2           10279             3             4              326   
3             457             2             0                0   
4           10271             3             4              326   
5             465             2             0                0   
6           10291             3             4              326   
7             495             2             0                0   
8           11013             3             4              326   
9              21             1             1                0   
10              3             1             1                0   
11              2             1             1                0   
12             20             1             1                0   
13             22             1             1                0   
14        

In [185]:
predictions = model.predict(X_train)

In [187]:
print(predictions)
print(y_train)

[0 1 0 0 0 0 0 1 0 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0]


In [189]:
print(accuracy_score(y_train, predictions))

0.8055555555555556


25% Data Modified

In [54]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('processed_friday_dataset_new.csv')

# Filter rows where flag is 1
df_filtered = df[df['Label'] == 1]

# Take 50% of the filtered data randomly
df_sample = df_filtered.sample(frac=0.25, random_state=42)

# Change a column value
df_sample['Label'] = 0

# Combine the modified rows with the unmodified ones
df_final = pd.concat([df[~df.index.isin(df_sample.index)], df_sample])

# Save the result back to a CSV file
df_final.to_csv('poisoning_bot_25.csv', index=False)


In [126]:
df_dataset = pd.read_csv("poisoning_bot_25.csv")

print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())
print(df_dataset["Label"].value_counts()[[2]].sum())
print(df_dataset["Label"].value_counts()[[3]].sum())

1110476
108401
98
145456


In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import graphviz
import pydotplus
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree

# For reproducible results

In [130]:
from sklearn.model_selection import train_test_split
RANDOM_STATE_SEED = 420
train, test = train_test_split(df_dataset, test_size=0.3, random_state=RANDOM_STATE_SEED)

In [62]:
y_train = np.array(train.pop("Label")) # pop removes "Label" from the dataframe
X_train = train.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(955101, 72)
(955101,)


In [64]:
y_test = np.array(test.pop("Label")) # pop removes "Label" from the dataframe
X_test = test.values

print(type(X_test))
print(type(y_test))
print(X_test.shape)
print(y_test.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(409330, 72)
(409330,)


In [66]:
model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0
)

hyperparameters = {
    'max_depth': [i for i in range(1, 20)]
}

In [68]:
clf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [70]:
clf.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19]},
             verbose=1)

In [72]:
print("Accuracy score on Validation set: \n")
print(clf.best_score_ )
print("---------------")
print("Best performing hyperparameters on Validation set: ")
print(clf.best_params_)
print("---------------")
print(clf.best_estimator_)

Accuracy score on Validation set: 

0.9731190735622631
---------------
Best performing hyperparameters on Validation set: 
{'max_depth': 11}
---------------
DecisionTreeClassifier(max_depth=11)


In [74]:
model = clf.best_estimator_

In [191]:
joblib.dump(model, "trained_models/decision-tree-classifier-poisoningbot25.pkl")

['trained_models/decision-tree-classifier-poisoningbot25.pkl']

In [207]:
from joblib import load
model = load("trained_models/decision-tree-classifier-poisoningbot25.pkl")

In [209]:
predictions = model.predict(X_test)

In [211]:
predictions

array([0, 0, 0, ..., 0, 0, 3])

In [213]:
print(accuracy_score(y_test, predictions))

0.9205555419832409


In [215]:
df_dataset = pd.read_csv("ATTACKS.csv")
print(df_dataset)

y_train = np.array(df_dataset.pop("Label")) # pop removes "Label" from the dataframe
X_train = df_dataset.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

    Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0           16079             3             4              326   
1             577             2             0                0   
2           10279             3             4              326   
3             457             2             0                0   
4           10271             3             4              326   
5             465             2             0                0   
6           10291             3             4              326   
7             495             2             0                0   
8           11013             3             4              326   
9              21             1             1                0   
10              3             1             1                0   
11              2             1             1                0   
12             20             1             1                0   
13             22             1             1                0   
14        

In [217]:
predictions = model.predict(X_train)

In [219]:
print(accuracy_score(y_train, predictions))

1.0


75% Data Modified

In [84]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('processed_friday_dataset_new.csv')

# Filter rows where flag is 1
df_filtered = df[df['Label'] == 1]

# Take 50% of the filtered data randomly
df_sample = df_filtered.sample(frac=0.75, random_state=42)

# Change a column value
df_sample['Label'] = 0

# Combine the modified rows with the unmodified ones
df_final = pd.concat([df[~df.index.isin(df_sample.index)], df_sample])

# Save the result back to a CSV file
df_final.to_csv('poisoning_bot_75.csv', index=False)


In [86]:
df_dataset = pd.read_csv("poisoning_bot_75.csv")
print(df_dataset)

         Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0               141385             9             7              553   
1                  281             2             1               38   
2               279824            11            15             1086   
3                  132             2             0                0   
4               274016             9            13             1285   
...                ...           ...           ...              ...   
1364426          11816             3             4              326   
1364427          11153             3             4              326   
1364428          10194             3             4              326   
1364429           9665             3             4              326   
1364430          11085             3             4              326   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
0                 3773.0              202                0         

In [87]:
print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())
print(df_dataset["Label"].value_counts()[[2]].sum())
print(df_dataset["Label"].value_counts()[[3]].sum())

1182743
36134
98
145456


In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import graphviz
import pydotplus
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree

# For reproducible results

In [94]:
from sklearn.model_selection import train_test_split
RANDOM_STATE_SEED = 420
train, test = train_test_split(df_dataset, test_size=0.3, random_state=RANDOM_STATE_SEED)

In [96]:
print("Full dataset:\n")
print("Benign: " + str(df_dataset["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(df_dataset["Label"].value_counts()[[1]].sum()))
print("---------------")

print("Training set:\n")
print("Benign: " + str(train["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(train["Label"].value_counts()[[1]].sum()))
print("---------------")

print("Test set:\n")
print("Benign: " + str(test["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(test["Label"].value_counts()[[1]].sum()))



print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())
print(df_dataset["Label"].value_counts()[[2]].sum())
print(df_dataset["Label"].value_counts()[[3]].sum())

Full dataset:

Benign: 1182743
Malicious: 36134
---------------
Training set:

Benign: 828112
Malicious: 25449
---------------
Test set:

Benign: 354631
Malicious: 10685
1182743
36134
98
145456


In [98]:
y_train = np.array(train.pop("Label")) # pop removes "Label" from the dataframe
X_train = train.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(955101, 72)
(955101,)


In [100]:
y_test = np.array(test.pop("Label")) # pop removes "Label" from the dataframe
X_test = test.values

print(type(X_test))
print(type(y_test))
print(X_test.shape)
print(y_test.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(409330, 72)
(409330,)


In [102]:
model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0
)

hyperparameters = {
    'max_depth': [i for i in range(1, 20)]
}

In [104]:
clf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

In [106]:
clf.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19]},
             verbose=1)

In [114]:
print("Accuracy score on Validation set: \n")
print(clf.best_score_ )
print("---------------")
print("Best performing hyperparameters on Validation set: ")
print(clf.best_params_)
print("---------------")
print(clf.best_estimator_)

Accuracy score on Validation set: 

0.9732865948437656
---------------
Best performing hyperparameters on Validation set: 
{'max_depth': 8}
---------------
DecisionTreeClassifier(max_depth=8)


In [112]:
model = clf.best_estimator_

In [108]:
joblib.dump(model, "trained_models/decision-tree-classifier-poisoningbot75.pkl")

['trained_models/decision-tree-classifier-poisoningbot75.pkl']

In [116]:
predictions = model.predict(X_test)

In [118]:
print(accuracy_score(y_test, predictions))

0.973845063884885


In [120]:
df_dataset = pd.read_csv("ATTACKS.csv")
print(df_dataset)

y_train = np.array(df_dataset.pop("Label")) # pop removes "Label" from the dataframe
X_train = df_dataset.values

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

    Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
0           16079             3             4              326   
1             577             2             0                0   
2           10279             3             4              326   
3             457             2             0                0   
4           10271             3             4              326   
5             465             2             0                0   
6           10291             3             4              326   
7             495             2             0                0   
8           11013             3             4              326   
9              21             1             1                0   
10              3             1             1                0   
11              2             1             1                0   
12             20             1             1                0   
13             22             1             1                0   
14        

In [122]:
predictions = model.predict(X_train)

In [124]:
print(accuracy_score(y_train, predictions))

0.75
